## Materials 
At the start, we have audio and annotated textgrids of **regilaul** songs, annotated for ictus/off-ictus and phrase text, then force-aligned using Praat's built in eSpeak forced aligner for Estonian to word and then segment. Then, we use the estnltk vabamorf package to syllabify the words so that we can annotate the textgrid further with syllable quantity (Estonian has 3) and whether or not it is accented at the word level. We end up with a dataframe containing the data from three of the(Interval) tiers of the textgrid, acquiring duration data for words, individual segments, and (eventually) syllables. 

In [6]:
import pandas as pd
import parselmouth
from estnltk.vabamorf.morf import syllabify_word
import tgt
import string
#test method on a single TextGrid:
gridDir2 = "/Users/sarah/Git/qp_final/songs/txtgridtest/55.TextGrid"


def get_duration_labels(textgrid, wordTier,s1,s2,ictusTier):
    tmp = tgt.read_textgrid(textgrid)
    words = tmp.get_tier_by_name(wordTier)
    firstSyll = tmp.get_tier_by_name(s1)
    secSyll = tmp.get_tier_by_name(s2)
    ictus = tmp.get_tier_by_name(ictusTier)
    segments = []
    wordlist = words.intervals
    
    for interval in wordlist:
        onset = interval.start_time
        offset = interval.end_time
       # wordms = offset-onset
        word = interval.text
        syllablist = syllabify_word(word,as_dict=True)

        i = 0
        #keep less than length of the list to avoid indexing errors on monosyllabic words
        while i < len(syllablist): 
            tmpsy = syllablist[i] 
            ortho = tmpsy.get('syllable')
            q = tmpsy.get('quantity')
            #quick lil hack to pull Q2 into the intercept
            #if q_tmp != 2: 
            #    q = q_tmp
            #else: q = 'a2'
            a = tmpsy.get('accent')

            if i == 0: 
                tmpinterval = firstSyll.get_annotations_between_timepoints(onset,offset)
            elif i == 1: 
                tmpinterval = secSyll.get_annotations_between_timepoints(onset,offset)
            #break out of the loop after getting the second syllable
            else: 
                i += 1 
                break
            #skip syllables with no annotations in the analysis tiers
            if len(tmpinterval)==0: 
                i+= 1 
                break
            for vowel in tmpinterval:
              
                segment = vowel.text
                vOnset = vowel.start_time
                vOffset = vowel.end_time
                vMidpoint = vOffset - (vOnset/2)
                dur = vOffset-vOnset
                tmpick = ictus.get_annotations_by_time(vMidpoint)
                if len(tmpick) > 0 :
                    ick = tmpick[0].text
                else: ick = "off"
                row = (word,ortho,i,segment,q,a,ick,dur,vMidpoint) 
                segments.append(row)
                
            i+= 1  
            
    nu_df = pd.DataFrame(segments,columns=["word","syll","index","segment","quantity","stressed","ictus","duration","midpoint"])
    return nu_df
    

# syl_dur_df = get_duration_labels(gridDir2,"word","word/phon","ictus")
# syl_dur_df.head()

onetwo_df = get_duration_labels(gridDir2,"word","s1","s2","ictus")
onetwo_df



,word,syll,index,segment,quantity,stressed,ictus,duration,midpoint
0,"Laula,",lau,0,a,2,1,ictus,0.156972,0.412077
1,"Laula,",lau,0,u,2,1,ictus,0.077000,0.410591
2,"Laula,","la,",1,a,2,0,ictus,0.162320,0.575411
3,"laula,",lau,0,a,2,1,ictus,0.111145,0.651171
4,"laula,",lau,0,u,2,1,ictus,0.099453,0.695051
5,"laula,","la,",1,a,2,0,off,0.163946,0.848972
6,"suukene,",suu,0,uː,2,1,ictus,0.311279,1.175601
7,"suukene,",ke,1,e,1,0,off,0.306746,1.418378
8,"liigu,",lii,0,iː,2,1,off,0.198803,2.272068
9,"liigu,","gu,",1,u,2,0,off,0.134310,2.350727


## Adding Spectral data
now that we have the duration data from the textgrid, we can query specific timepoints for information about the acoustic signal. The following function uses the midpoint (which we snagged while we were making the dataframe above) and get the first three formants(Hz) for each segment. 

In [7]:

import parselmouth

test = "songs/matchgrids/55.wav"

def get_formants(syl_dur_df, wave):
    song = parselmouth.Sound(wave)
    formant = song.to_formant_burg()
    f1 = []
    f2 = []
    euc = []
    for float in syl_dur_df.midpoint:
        time = float
        first = formant.get_value_at_time(1,time)
        f1.append(first)
        second = formant.get_value_at_time(2, time)
        f2.append(second)
        euc.append(second-first)
    syl_dur_df["f1"] = f1
    syl_dur_df["f2"] = f2
    syl_dur_df["euc"] = euc
    return syl_dur_df
nu_df = get_formants(onetwo_df,test)
nu_df.head()


,word,syll,index,segment,quantity,stressed,ictus,duration,midpoint,f1,f2,euc
0,"Laula,",lau,0,a,2,1,ictus,0.156972,0.412077,441.456410,1674.755725,1233.299315
1,"Laula,",lau,0,u,2,1,ictus,0.077000,0.410591,438.833336,1640.944799,1202.111463
2,"Laula,","la,",1,a,2,0,ictus,0.162320,0.575411,563.918963,1033.169460,469.250497
3,"laula,",lau,0,a,2,1,ictus,0.111145,0.651171,521.254933,977.549779,456.294845
4,"laula,",lau,0,u,2,1,ictus,0.099453,0.695051,546.932042,933.367997,386.435954


In [8]:
from os.path import join
#runs a for loop over a directory using the above-specified functions

test = "songs/txtgridtest"
songs = "songs/matchgrids"

for fn in os.listdir(test):
    if '.TextGrid' not in fn: 
        continue 
    n = fn.strip('.TextGrid') 
    wave = join(songs, n + '.wav')
    data_file = open(  n +"_v_dur_space.csv",'w')
    #make a dataframe with the interval tiers of the textgrid
    tmp = pd.DataFrame(get_duration_labels(join(test,fn), "word","s1","s2","ictus"))
    #add the formant data to the dataframe
    nu_df = get_formants(tmp,wave)
    #print(nu_df.head())
    nu_df.to_csv(data_file)
    data_file.close()

# Now we put it into a big pile!

Here we concatenate all the data we have so far into one large pandas dataframe. At this point, we can keep annotating songs for the corpus, and as textgrids are finished we can run the scripts above to add them into the larger dataset. We're also gonna take the opportunity to add some metadata to the dataframes: fileid(song) and performer initials as potential grouping factors. 

In [9]:

import os 
import pandas as pd 
import statsmodels.formula.api as smf
folder = "feline_nottiger"
meta =  pd.read_csv("songs/song_metadata.csv")


songs_dfs = []
for fn in os.listdir(folder):
    if '.csv' not in fn: continue
    whole_name = os.path.join(folder,fn)
    song_df = pd.read_csv(whole_name)
    fileid1 = fn.strip('_v_dur_space.csv')
    fileid = int(fileid1)
    row = meta.index[meta['track'] == fileid].tolist()
    if len(row) !=0 :
        performer = meta.performer[row[0]]
    else: performer = "couldn't get match"
    for index in song_df:
        song_df['fileid'] = fileid
        song_df['performer'] = performer

        
    songs_dfs.append(song_df)

big_frame = pd.concat(songs_dfs, ignore_index=True)
big_frame.describe()

# big_frame
#clean_frame = pd.DataFrame(big_frame[['quantity','stress','segment','seg_duration','ictus','euc','fileid','performer']])
#clean_frame.head()

big_frame

,Unnamed: 0,word,syll,index,segment,quantity,stressed,ictus,duration,midpoint,f1,f2,euc,fileid,performer
0,0,miks,miks,0,i,3,1,ictus,0.103965,1.423199,606.835891,1313.885761,707.049869,94,LK
1,1,sa,sa,0,a,3,1,ictus,0.125341,1.581561,674.869881,1469.899762,795.029880,94,LK
2,2,mullu,mul,0,ul,2,1,off,0.159480,1.728757,754.864294,1336.479715,581.615421,94,LK
3,3,mullu,lu,1,u,1,0,off,0.158085,1.844135,590.123359,1120.789725,530.666366,94,LK
4,4,meile(e)i,mei,0,ei,2,1,ictus,0.235947,2.030942,711.517330,1380.456361,668.939031,94,LK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,22,pani,pa,0,a,1,1,ictus,0.113842,37.244394,660.363937,1572.385981,912.022044,77,LO
606,23,pani,ni,1,ɪ,1,0,ictus,0.053486,37.260037,837.706232,1943.857180,1106.150949,77,LO
607,24,"taevudes,",tae,0,a,2,1,ictus,0.192910,39.048550,1061.396268,1371.303953,309.907685,77,LO
608,25,"taevudes,",tae,0,e,2,1,ictus,0.139361,39.091456,1053.974515,1800.978853,747.004339,77,LO


In [12]:
big_frame['index'] = big_frame['index'].astype(object)
big_frame['quantity'] = big_frame['quantity'].astype(object)
big_frame['stressed'] = big_frame['stressed'].astype(object)
big_frame['fileid'] = big_frame['fileid'].astype(object)

corpus_data = open('bu_all_songs.csv','w')
big_frame.to_csv(corpus_data)
corpus_data.close()
big_frame.dtypes

Unnamed: 0      int64
word           object
syll           object
index          object
segment        object
quantity       object
stressed       object
ictus          object
duration      float64
midpoint      float64
f1            float64
f2            float64
euc           float64
fileid         object
performer      object
dtype: object

In [1]:
import statsmodels.api as sm
data = big_frame[["ictus","duration"]]
table = sm.stats.Table.from_data(data)

tab = pd.crosstab(big_frame["ictus"],big_frame["stressed"])

tab

NameError: name 'big_frame' is not defined

In [7]:
big_frame.groupby('stressed')['duration'].describe()

,count,mean,std,min,25%,50%,75%,max
stressed,,,,,,,,
0,271.0,0.186972,0.071192,0.035638,0.138025,0.182604,0.227726,0.412773
1,339.0,0.218175,0.097679,0.025764,0.148371,0.206735,0.277601,0.570958


In [8]:
dummy_data = pd.get_dummies(big_frame,columns = ["quantity"], drop_first=False)
#dummy_data.drop('quantity_2',axis=1,inplace=True)
dummy_data.head()

/Users/sarah/miniconda3/envs/soundmines/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


,Unnamed: 0,word,syll,index,segment,stressed,ictus,duration,midpoint,f1,f2,euc,fileid,performer,quantity_1,quantity_2,quantity_3
0,0,miks,miks,0,i,1,ictus,0.103965,1.423199,606.835891,1313.885761,707.049869,94,LK,0,0,1
1,1,sa,sa,0,a,1,ictus,0.125341,1.581561,674.869881,1469.899762,795.029880,94,LK,0,0,1
2,2,mullu,mul,0,ul,1,off,0.159480,1.728757,754.864294,1336.479715,581.615421,94,LK,0,1,0
3,3,mullu,lu,1,u,0,off,0.158085,1.844135,590.123359,1120.789725,530.666366,94,LK,1,0,0
4,4,meile(e)i,mei,0,ei,1,ictus,0.235947,2.030942,711.517330,1380.456361,668.939031,94,LK,0,1,0


In [9]:
q1 = smf.ols('duration ~ C(quantity, Treatment(reference=2))',data = big_frame).fit()
q1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               duration   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.024
Date:                Sun, 31 Jul 2022   Prob (F-statistic):              0.360
Time:                        14:08:38   Log-Likelihood:                 616.99
No. Observations:                 610   AIC:                            -1228.
Df Residuals:                     607   BIC:                            -1215.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                    0.1990      0.005     38.479      0.000       0.189       0.209
C(quantity, Treatment(reference=2))[T.1]     0.0097      0.008      1.251      0.211      -0.006       0.025
C(quantity, Treatment(reference=2))[T.3]     0.0115      0.011      1.055      0.292      -0.010       0.033
==============================================================================
Omnibus:                       40.450   Durbin-Watson:                   1.400
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               47.030
Skew:                           0.636   Prob(JB):                     6.13e-11
Kurtosis:                       3.483   Cond. No.                         3.63
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [35]:
q_ick_dur_model = smf.ols('duration ~ C(ictus, Treatment(reference = "ictus")) + C(quantity, Treatment(reference = 2))',data = big_frame).fit()
print(q_ick_dur_model.summary())

                            OLS Regression Results                            
Dep. Variable:               duration   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.968
Date:                Sun, 31 Jul 2022   Prob (F-statistic):              0.118
Time:                        17:33:52   Log-Likelihood:                 618.92
No. Observations:                 610   AIC:                            -1230.
Df Residuals:                     606   BIC:                            -1212.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                                    coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------

In [11]:
q_dur_model = smf.ols('duration ~ C(quantity, Treatment(reference = 2)) + C(stressed, Treatment(reference = 1))',data = big_frame).fit()
q_dur_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               duration   R-squared:                       0.046
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     9.750
Date:                Sun, 31 Jul 2022   Prob (F-statistic):           2.73e-06
Time:                        14:08:38   Log-Likelihood:                 630.34
No. Observations:                 610   AIC:                            -1253.
Df Residuals:                     606   BIC:                            -1235.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                    0.2142      0.006     36.657      0.000       0.203       0.226
C(quantity, Treatment(reference=2))[T.1]     0.0238      0.008      2.953      0.003       0.008       0.040
C(quantity, Treatment(reference=2))[T.3]    -0.0036      0.011     -0.329      0.742      -0.025       0.018
C(stressed, Treatment(reference=1))[T.0]    -0.0417      0.008     -5.207      0.000      -0.057      -0.026
==============================================================================
Omnibus:                       29.911   Durbin-Watson:                   1.226
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               34.174
Skew:                           0.508   Prob(JB):                     3.79e-08
Kurtosis:                       3.560   Cond. No.                         4.21
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
import pandas as pd 
import statsmodels .formula.api as smf
ictus_dur = open('ictus_durs.txt','w')
ictus_dur.close()
#is ictus (song prominence) a good predictor for vowel duration?

ickmodel = smf.ols('duration ~ C(ictus, Treatment(reference= "off"))', data=big_frame).fit()


ickmodel.summary()

#off-ictus predicts shorter duration (negative slope) p< 0.05


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               duration   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     4.253
Date:                Sun, 31 Jul 2022   Prob (F-statistic):             0.0396
Time:                        14:08:38   Log-Likelihood:                 618.09
No. Observations:                 610   AIC:                            -1232.
Df Residuals:                     608   BIC:                            -1223.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
Intercept                                         0.1985      0.005     43.624      0.000       0.190       0.207
C(ictus, Treatment(reference="off"))[T.ictus]     0.0151      0.007      2.062      0.040       0.001       0.029
==============================================================================
Omnibus:                       36.754   Durbin-Watson:                   1.393
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               41.905
Skew:                           0.607   Prob(JB):                     7.95e-10
Kurtosis:                       3.417   Cond. No.                         2.44
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
stressdur =  smf.ols('duration ~ stressed', data=big_frame).fit()
stressdur.summary()
#stress is a predictor of duration: stressed vowels p< 0.0


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               duration   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     19.41
Date:                Sun, 31 Jul 2022   Prob (F-statistic):           1.25e-05
Time:                        14:08:38   Log-Likelihood:                 625.54
No. Observations:                 610   AIC:                            -1247.
Df Residuals:                     608   BIC:                            -1238.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.1870      0.005     35.412      0.000       0.177       0.197
stressed[T.1]     0.0312      0.007      4.406      0.000       0.017       0.045
==============================================================================
Omnibus:                       28.176   Durbin-Watson:                   1.249
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               30.942
Skew:                           0.520   Prob(JB):                     1.91e-07
Kurtosis:                       3.368   Cond. No.                         2.77
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
bothdur =  smf.ols('duration ~ stressed + C(ictus, Treatment(reference= "off"))', data=big_frame).fit()
bothdur.summary()
#with unstressed off-ictus as intercept, both stress and off-ictus are significant predictos of vowel duration
#AIC of ictus only: -1232.
#AIC of stress only: -1247
#AIC of mixed: -1250 
#stress correlates with an increase in vowel duration
#ictus correlates with an increase in vowel duration

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               duration   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     12.09
Date:                Sun, 31 Jul 2022   Prob (F-statistic):           7.08e-06
Time:                        14:08:38   Log-Likelihood:                 627.88
No. Observations:                 610   AIC:                            -1250.
Df Residuals:                     607   BIC:                            -1237.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
Intercept                                         0.1808      0.006     30.221      0.000       0.169       0.193
stressed[T.1]                                     0.0314      0.007      4.450      0.000       0.018       0.045
C(ictus, Treatment(reference="off"))[T.ictus]     0.0156      0.007      2.159      0.031       0.001       0.030
==============================================================================
Omnibus:                       24.467   Durbin-Watson:                   1.246
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               26.340
Skew:                           0.486   Prob(JB):                     1.91e-06
Kurtosis:                       3.300   Cond. No.                         3.15
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [33]:
stressspace =  smf.mixedlm('euc ~ stressed', data=big_frame,groups = 'performer').fit()
stressspace.summary()
#when we group by performer, stress has a positive slope, but the effect is not significant

<class 'statsmodels.iolib.summary2.Summary'>
"""
            Mixed Linear Model Regression Results
=============================================================
Model:              MixedLM  Dependent Variable:  euc        
No. Observations:   610      Method:              REML       
No. Groups:         3        Scale:               155574.0033
Min. group size:    76       Log-Likelihood:      -4505.5926 
Max. group size:    303      Converged:           Yes        
Mean group size:    203.3                                    
-------------------------------------------------------------
                Coef.   Std.Err.   z    P>|z|  [0.025  0.975]
-------------------------------------------------------------
Intercept       826.200   77.748 10.627 0.000 673.818 978.583
stressed[T.1]     0.855   32.152  0.027 0.979 -62.163  63.873
performer Var 16093.814   42.472                             
=============================================================

"""

In [32]:
ickspace =  smf.mixedlm('euc ~ ictus', data=big_frame, groups = 'performer').fit()
ickspace.summary()

#interesting! off-ictus has a positive slope. not signifiant. we could call this `approaching significance' `



<class 'statsmodels.iolib.summary2.Summary'>
"""
            Mixed Linear Model Regression Results
=============================================================
Model:              MixedLM  Dependent Variable:  euc        
No. Observations:   610      Method:              REML       
No. Groups:         3        Scale:               154714.9841
Min. group size:    76       Log-Likelihood:      -4503.7722 
Max. group size:    303      Converged:           Yes        
Mean group size:    203.3                                    
-------------------------------------------------------------
                Coef.   Std.Err.   z    P>|z|  [0.025  0.975]
-------------------------------------------------------------
Intercept       790.660   74.324 10.638 0.000 644.987 936.332
ictus[T.off]     63.191   33.447  1.889 0.059  -2.364 128.746
performer Var 14434.113   38.399                             
=============================================================

"""

In [31]:
bothspace =  smf.mixedlm('euc ~ C(ictus, Treatment(reference = "off")) + C(stressed, Treatment(reference=0))', data=big_frame,groups = 'performer').fit()
bothspace.summary()


<class 'statsmodels.iolib.summary2.Summary'>
"""
                             Mixed Linear Model Regression Results
===============================================================================================
Model:                          MixedLM             Dependent Variable:             euc        
No. Observations:               610                 Method:                         REML       
No. Groups:                     3                   Scale:                          154970.6507
Min. group size:                76                  Log-Likelihood:                 -4499.3850 
Max. group size:                303                 Converged:                      Yes        
Mean group size:                203.3                                                          
-----------------------------------------------------------------------------------------------
                                                Coef.   Std.Err.   z    P>|z|  [0.025   0.975] 
-----------------------------------------------------------------------------------------------
Intercept                                       853.939   75.521 11.307 0.000  705.921 1001.957
C(ictus, Treatment(reference="off"))[T.ictus]   -63.196   33.479 -1.888 0.059 -128.814    2.421
C(stressed, Treatment(reference=0))[T.1]         -0.149   32.094 -0.005 0.996  -63.053   62.755
performer Var                                 14434.003   38.369                               
===============================================================================================

"""

In [18]:
stressmodel = smf.ols("duration ~ stressed", data=big_frame).fit()
print(stressmodel.summary())

#stress is a predictor of longer vowel duration

                            OLS Regression Results                            
Dep. Variable:               duration   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     19.41
Date:                Sun, 31 Jul 2022   Prob (F-statistic):           1.25e-05
Time:                        14:08:38   Log-Likelihood:                 625.54
No. Observations:                 610   AIC:                            -1247.
Df Residuals:                     608   BIC:                            -1238.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.1870      0.005     35.412